# generation

In [1]:
%run load_model.py

In [2]:
import os
import torch.nn.functional as F
from torch.autograd import Variable

# load word list
with open(os.path.join('embedding', 'word_dict_co100.txt')) as f:
    known_words = f.read().strip().split()
    vocab_size = len(known_words)
word_to_index = lambda word: known_words.index(word) if word in known_words else known_words.index('<ukn>')
index_to_word = lambda ind: known_words[ind]

In [3]:
N = 5
sent_list = ['<s>'] + ['تیم', 'رئال']

sentences = []
for i in range(100):
    counter = len(sent_list)
    seq_ind = [word_to_index(i) for i in sent_list]
    word_ind = 0
    while counter < 100 and word_ind != word_to_index('</s>'):
        v = Variable(torch.LongTensor(seq_ind).view(counter, 1))
        result = model.forward(v)
        top_N_score, top_N_ind = torch.topk(result[-1], N)
        top_N_ind = top_N_ind.squeeze_().cpu().data.numpy()
        top_N_prob = F.softmax(top_N_score, dim=1).squeeze_().cpu().data.numpy()
        word_ind = np.random.choice(top_N_ind, p=top_N_prob)
        seq_ind.append(int(word_ind))
        counter += 1
    sentences.append(seq_ind[1:-1])

with open('generated.txt', 'w+') as f:
    f.write('\n'.join([' '.join(map(lambda x: index_to_word(x), seq)) for seq in sentences]))